In [1]:
from ScanImageTiffReader import ScanImageTiffReader
import json
import ast
from PIL import Image
import os
import tifffile
import numpy as np
import h5py

In [10]:
mc_img_folder = r'Z:\TM_Lab\Edward\Cerebellum_Imaging\O_L3\20240326\acq2\suite2p\plane0\reg_tif' # folder containing fragments of motion corrected tif from suite2p
#test_img_folder = r'Z:\TM_Lab\Edward\Cerebellum_Imaging\O_L3\20240326\acq2\test_tif' # folder containing fragments of motion corrected tif from suite2p

In [2]:
def crop_or_pad(image, target_width, target_height):
    """
    Pad or crop the image to make it 512x512.
    """
    # Crop if larger
    if image.shape[0] > target_height or image.shape[1] > target_width:
        # Center crop to 512x512
        startx = image.shape[1]//2 - (target_width//2)
        starty = image.shape[0]//2 - (target_height//2)    
        image = image[starty:starty+target_height, startx:startx+target_width]
    # Pad if smaller
    if image.shape[0] < target_height or image.shape[1] < target_width:
        # Calculate padding sizes
        pad_x = (target_width - image.shape[1]) // 2
        pad_y = (target_height - image.shape[0]) // 2
        image = np.pad(image, [(pad_y, target_height - image.shape[0] - pad_y), (pad_x, target_width - image.shape[1] - pad_x)], mode='constant')
    return image
def concatenate_tiffs_for_deep_interp(folder_path, num_roi):
    # target output, 512x512
    tiff_files = [f for f in os.listdir(folder_path) if f.endswith('.tif')]
    tiff_files.sort()  # Ensure files are in the desired order

    if num_roi == 1:
        all_images = []

        for tiff_file in tiff_files:
            with tifffile.TiffFile(os.path.join(folder_path, tiff_file)) as tif:
                for page in tif.pages:
                    image = page.asarray()
                    # Adjust each image to be 512x512
                    processed_image = crop_or_pad(image, 512, 512)
                    all_images.append(processed_image)

        # Save the adjusted images as a new TIFF file
        output_file_path = os.path.join(folder_path, 'concatenated_adjusted_output.tif')
        with tifffile.TiffWriter(output_file_path, bigtiff=True) as tif:
            for image in all_images:
                tif.write(image)

    elif num_roi == 2:
        all_top_images = []
        all_bottom_images = []
        for tiff_file in tiff_files:
            with tifffile.TiffFile(os.path.join(folder_path, tiff_file)) as tif:
                for page in tif.pages:
                    image = page.asarray()
                    # Adjust each image to be 512x512
                    processed_image = crop_or_pad(image, 512, 1024)
                    all_top_images.append(processed_image[:512, :])
                    all_bottom_images.append(processed_image[512:, :])

        # Save the adjusted images as a new TIFF file
        output_file_path1 = os.path.join(folder_path, 'concatenated_top_images.tif')
        output_file_path2 = os.path.join(folder_path, 'concatenated_bottom_images.tif')
        with tifffile.TiffWriter(output_file_path1, bigtiff=True) as tif:
            for image in all_top_images:
                tif.write(image)
        with tifffile.TiffWriter(output_file_path2, bigtiff=True) as tif:
            for image in all_bottom_images:
                tif.write(image)
    

In [4]:
concatenate_tiffs_for_deep_interp(test_img_folder, num_roi=2)
#concatenate_tiffs_for_deep_interp(mc_img_folder, num_roi=2)

In [3]:
def convert_h5_to_tif(h5_file_path, tif_file_path, dataset_name):
    """
    Convert a dataset from an HDF5 file to a TIFF file.

    Parameters:
    - h5_file_path: Path to the input HDF5 file.
    - tif_file_path: Path to the output TIFF file.
    - dataset_name: Name of the dataset within the HDF5 file to convert.
    """
    with h5py.File(h5_file_path, 'r') as h5_file:
        # Access the specified dataset in the HDF5 file
        dataset = h5_file[dataset_name][()]
        
        # Save the dataset as a TIFF file
        tifffile.imwrite(tif_file_path, dataset, photometric='minisblack')

In [12]:
h5_file_path = r"Z:\TM_Lab\Edward\Cerebellum_Imaging\O_L3\20240326\acq2\suite2p\plane0\reg_tif\concatenated_bottom_images_interped.h5"
tif_file_path = r"Z:\TM_Lab\Edward\Cerebellum_Imaging\O_L3\20240326\acq2\suite2p\plane0\reg_tif\concatenated_bottom_images_interped.tif"
convert_h5_to_tif(h5_file_path, tif_file_path, "data")

In [4]:
def concatenate_tiffs_left_right_roi(file1, file2, output_file):
    with tifffile.TiffFile(file1) as tif1, tifffile.TiffFile(file2) as tif2:
        # Load the images
        img1 = tif1.asarray()
        img2 = tif2.asarray()
        
        # Check if the heights are the same
        if img1.shape[0] == img2.shape[0]:
            # Concatenate along the width dimension
            concatenated_img = np.concatenate((img1, img2), axis=1)
            
            # Save the concatenated image
            tifffile.imwrite(output_file, concatenated_img)
            print(f"Concatenated TIFF saved as {output_file}.")
        else:
            print("The TIFF files do not have the same height and cannot be concatenated.")

In [5]:
left_roi_tif = r"Z:\TM_Lab\Edward\Cerebellum_Imaging\O_L3\20240326\acq2\suite2p\plane0\reg_tif\concatenated_top_images_interped.tif"
right_roi_tif = r"Z:\TM_Lab\Edward\Cerebellum_Imaging\O_L3\20240326\acq2\suite2p\plane0\reg_tif\concatenated_bottom_images_interped.tif"
outfile = r"Z:\TM_Lab\Edward\Cerebellum_Imaging\O_L3\20240326\acq2\deepinterp_img\20240326_acq2_interped.tif"
concatenate_tiffs_left_right_roi(left_roi_tif, right_roi_tif, outfile)

FileNotFoundError: [Errno 2] No such file or directory: 'Z:\\TM_Lab\\Edward\\Cerebellum_Imaging\\O_L3\\20240326\\acq2\\deepinterp_img\\20240326_acq2_interped.tif'

In [2]:
img_path = r'H:\Cerebellum_Imaging\Q_R2\20240320\acq1\acq1_hab1_2roi_15hz_superficial_00001.tif'
reader=ScanImageTiffReader(img_path)

Exception: Could not open file.

In [24]:
vol=reader.data()
print('Frame: %s Height: %s Width: %s' %(vol.shape[0], vol.shape[1], vol.shape[2]))

Frame: 14317 Height: 1040 Width: 512


In [27]:
frames_metadata = []

with ScanImageTiffReader(img_path) as reader:
    num_frames = reader.shape()[0]  # Assuming the first dimension represents frames
    
    for i in range(num_frames):
        description = reader.description(i)
        frame_metadata = {}  # Dictionary to hold this frame's metadata
        
        for line in description.split('\n'):
            if '=' in line:  # Ensure there's a key-value pair to parse
                key, value = line.split('=', 1)  # Split by the first '='
                key = key.strip()
                
                try:
                    # Safely evaluate the value string to the appropriate Python data type
                    # This works for numbers, lists, dictionaries, etc.
                    value = ast.literal_eval(value.strip())
                except (ValueError, SyntaxError):
                    # If evaluation fails, keep the value as a string
                    value = value.strip()
                
                frame_metadata[key] = value
        
        frames_metadata.append(frame_metadata)


Exception: Could not open file.

In [37]:
frames_metadata[0]

{'frameNumbers': 1,
 'acquisitionNumbers': 1,
 'frameNumberAcquisition': 1,
 'frameTimestamps_sec': 0.0,
 'acqTriggerTimestamps_sec': -0.00011737,
 'nextFileMarkerTimestamps_sec': -1.0,
 'endOfAcquisition': 0,
 'endOfAcquisitionMode': 0,
 'dcOverVoltage': 0,
 'epoch': '[2024  3  5 12 25 15.238]',
 'auxTrigger0': [],
 'auxTrigger1': [],
 'auxTrigger2': [],
 'auxTrigger3': [],
 'I2CData': {}}

In [38]:
all_auxTrigger0 = []

# Iterate through each frame's metadata
for frame_metadata in frames_metadata:
    # Check if 'auxTrigger0' exists and is not empty
    if 'auxTrigger0' in frame_metadata and frame_metadata['auxTrigger0']:
        # Extend the all_auxTrigger0 list with the numbers from this frame's auxTrigger0
        all_auxTrigger0.extend(frame_metadata['auxTrigger0'])

In [39]:
all_auxTrigger0

[33.78155021,
 45.68120846,
 46.73831696,
 57.58207253,
 58.64988146,
 67.60462751,
 68.66533688,
 76.1923746,
 77.24698399,
 89.06634541,
 90.20465545,
 99.51650445,
 100.57881332,
 106.30794343,
 107.3560526,
 119.87042058,
 120.94502926,
 130.54347846,
 131.60688759,
 144.02305433,
 145.0975635,
 154.54891217,
 155.61202118,
 167.29938358,
 168.37039251,
 176.15663213,
 177.21324122,
 187.68489387,
 188.75160327]

In [34]:
with ScanImageTiffReader(img_path) as reader:
    description = reader.description(1)
    print(type(description))
    print(description)
    print(reader.shape()[0])

<class 'str'>
frameNumbers = 2
acquisitionNumbers = 1
frameNumberAcquisition = 2
frameTimestamps_sec = 0.065715060
acqTriggerTimestamps_sec = -1.000000000
nextFileMarkerTimestamps_sec = -1.000000000
endOfAcquisition = 0
endOfAcquisitionMode = 0
dcOverVoltage = 0
epoch = [2024  3  5 12 25 15.238]
auxTrigger0 = []
auxTrigger1 = []
auxTrigger2 = []
auxTrigger3 = []
I2CData = {}
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [ ]:
with ScanImageTiffReader(img_path) as reader:
    o=json.loads(reader.metadata())
    print(o["RoiGroups"]["imagingRoiGroup"]["rois"]["scanfields"]["affine"])